In [ ]:
# Import required packages
%matplotlib inline
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import h5py
import xarray as xr
import rioxarray 
import datetime as dt
import hvplot.xarray
import hvplot.pandas
import holoviews as hv

import cartopy.crs as ccrs
from xrspatial.classify import natural_breaks

In [ ]:
waterbodies = gpd.read_file('datasets/GSI_Waterbodies.shp')


In [ ]:
waterbodies.plot()

# Clip to a required extent

In [ ]:
dsW = xr.load_dataset('datasets/indices/mndwi_1987_2022.nc')

In [ ]:
dsW.mndwi.mean('time').hvplot()

In [ ]:
from shapely.geometry import box
riv_geometry = [box(float(dsW.x.min().data), 
                    float(dsW.y.min().data), float(dsW.x.max().data), float(dsW.y.max().data))]
poly_gdf = gpd.GeoDataFrame([1], geometry=riv_geometry, crs=dsW.rio.crs)
waterbodies_if = waterbodies.clip(poly_gdf)

waterbodies_if.plot()

# More clipping

In [ ]:
cities = gpd.read_file('datasets/BIHAR_MAJOR_TOWNS.shp').to_crs("EPSG:4326")

In [ ]:
cities_clip = cities.cx[85.17:86.9,24.8:26.8]

In [ ]:
cities.hvplot(geo=True, tiles='OSM')*cities_clip.hvplot(geo=True)

# Coustom point

In [ ]:
ref = pd.DataFrame({'longitude': [86.123715], 'latitude': [25.416298], 'name':'Begusarai'})
reference_point  =   gpd.GeoDataFrame(
    ref, geometry=gpd.points_from_xy(ref.longitude, ref.latitude, crs="EPSG:4326"))

In [ ]:
cities.hvplot(geo=True)*cities_clip.hvplot(geo=True)*reference_point.hvplot(geo=True)

In [ ]:
waterbodies_if.hvplot(geo=True)*\
cities.hvplot(geo=True)*cities_clip.hvplot(geo=True)*reference_point.hvplot(geo=True)

In [ ]:
# Make polygons from csv-stored coordinates

In [ ]:
bbox_df = pd.read_csv('datasets/bboxes_stablePoint_WLs.csv')

In [ ]:
from shapely.geometry import box

geometry = [box(x1, y1, x2, y2) for x1,y1,x2,y2 in zip(bbox_df.x1, bbox_df.y1, bbox_df.x2, bbox_df.y2)]



In [ ]:
bbox_df = bbox_df.drop(['x1', 'x2', 'y1', 'y2'], axis=1)
bbox_geodf = gpd.GeoDataFrame(bbox_df, geometry=geometry)

print(bbox_geodf.head())

In [ ]:
bbox_geodf.hvplot(geo=True, alpha=0.4, c='r',tiles='OSM', width = 500, height = 700)

In [ ]:
bbox_geodf.to_file('bbox.shp', driver='ESRI Shapefile')